# 01 Make Base Standard Network

In this notebook we will run through:

1. Read in wrangler standard network  
2. Append centroid and centroid connectors
3. Calculate lanes
4. Export the network pickle

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import MTC

import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

# Remote I/O

In [10]:
input_dir = "../data/interim/step8_standard_format"
output_dir = "../data/processed/version_01"
lasso_dir = "../../Lasso"

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-21 13:41:10, INFO: Lasso base directory set as: ../../Lasso
2020-10-21 13:41:10, INFO: Lasso base directory set as: ../../Lasso


In [7]:
parameters.int_col

['model_link_id',
 'model_node_id',
 'A',
 'B',
 'drive_access',
 'walk_access',
 'bike_access',
 'truck_access',
 'ML_lanes_AM',
 'ML_lanes_MD',
 'ML_lanes_PM',
 'ML_lanes_EV',
 'ML_lanes_EA',
 'segment_id',
 'managed',
 'bus_only',
 'rail_only',
 'ft',
 'lanes_AM',
 'lanes_MD',
 'lanes_PM',
 'lanes_EA',
 'lanes_EV',
 'useclass_AM',
 'useclass_EA',
 'useclass_MD',
 'useclass_PM',
 'useclass_EV',
 'tollseg',
 'tollbooth']

In [8]:
parameters.properties_to_split

{'lanes': {'v': 'lanes',
  'time_periods': {'EA': ('3:00', '6:00'),
   'AM': ('6:00', '10:00'),
   'MD': ('10:00', '15:00'),
   'PM': ('15:00', '19:00'),
   'EV': ('19:00', '3:00')}},
 'ML_lanes': {'v': 'ML_lanes',
  'time_periods': {'EA': ('3:00', '6:00'),
   'AM': ('6:00', '10:00'),
   'MD': ('10:00', '15:00'),
   'PM': ('15:00', '19:00'),
   'EV': ('19:00', '3:00')}},
 'useclass': {'v': 'useclass',
  'time_periods': {'EA': ('3:00', '6:00'),
   'AM': ('6:00', '10:00'),
   'MD': ('10:00', '15:00'),
   'PM': ('15:00', '19:00'),
   'EV': ('19:00', '3:00')}}}

# Read Roadway and Transit Networks

In [11]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_file = link_file, 
    node_file = node_file, 
    shape_file = shape_file
)

2020-10-21 13:42:31, INFO: Reading from following files:
-../data/interim/step8_standard_format\link.json
-../data/interim/step8_standard_format\node.geojson
-../data/interim/step8_standard_format\shape.geojson.
2020-10-21 13:42:31, INFO: Reading from following files:
-../data/interim/step8_standard_format\link.json
-../data/interim/step8_standard_format\node.geojson
-../data/interim/step8_standard_format\shape.geojson.
2020-10-21 13:45:34, INFO: Read 1634769 links from ../data/interim/step8_standard_format\link.json
2020-10-21 13:45:34, INFO: Read 1634769 links from ../data/interim/step8_standard_format\link.json
2020-10-21 13:45:34, INFO: Read 644475 nodes from ../data/interim/step8_standard_format\node.geojson
2020-10-21 13:45:34, INFO: Read 644475 nodes from ../data/interim/step8_standard_format\node.geojson
2020-10-21 13:45:34, INFO: Read 869970 shapes from ../data/interim/step8_standard_format\shape.geojson
2020-10-21 13:45:34, INFO: Read 869970 shapes from ../data/interim/step8_

In [12]:
roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

transit_net = TransitNetwork.read(feed_path = input_dir)

# Make Travel Model Network

In [13]:
base_roadway_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = roadway_net, 
    parameters = parameters)

2020-10-21 13:54:27, INFO: Lasso base directory set as: ../../Lasso
2020-10-21 13:54:27, INFO: Lasso base directory set as: ../../Lasso


In [14]:
print(base_roadway_net.links_df.shape)
print(base_roadway_net.shapes_df.shape)
print(base_roadway_net.nodes_df.shape)

(1634769, 26)
(869970, 7)
(644475, 11)


In [15]:
base_roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

In [16]:
roadway_net.nodes_df.head(3)

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry
model_node_id_idx,,,,,,,,,,,
3000000,2.401245e+09,505d64eb98f1da8d812a3b3801034308,Contra Costa,1,1,1,3000000,0,-122.331554,37.981204,POINT (-122.33155 37.98120)
3000001,5.783907e+07,473979c78435732f01ca5a168afb62e0,Contra Costa,1,1,1,3000001,0,-121.944770,37.953322,POINT (-121.94477 37.95332)
5000000,1.024389e+09,fc7b575d5d8c961d4a70fca846ae7f80,Marin,1,1,1,5000000,0,-122.539828,37.897999,POINT (-122.53983 37.89800)


## Add centroid and centroid connector database

In [17]:
base_roadway_net = MTC.add_centroid_and_centroid_connector(
    roadway_network_object = base_roadway_net
)

2020-10-21 13:54:28, INFO: Adding centroid and centroid connector to standard network
2020-10-21 13:54:28, INFO: Adding centroid and centroid connector to standard network
2020-10-21 13:54:42, INFO: Finished adding centroid and centroid connectors
2020-10-21 13:54:42, INFO: Finished adding centroid and centroid connectors


In [18]:
type(base_roadway_net.parameters)

lasso.parameters.Parameters

In [19]:
print(base_roadway_net.links_df.shape)
print(base_roadway_net.shapes_df.shape)
print(base_roadway_net.nodes_df.shape)

(2221035, 26)
(1163103, 7)
(688952, 11)


In [20]:
base_roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

In [21]:
base_roadway_net.nodes_df.columns

Index(['osm_node_id', 'shst_node_id', 'county', 'drive_access', 'walk_access',
       'bike_access', 'model_node_id', 'rail_only', 'X', 'Y', 'geometry'],
      dtype='object')

## Calculate number of lanes

In [22]:
base_roadway_net = MTC.determine_number_of_lanes(
    roadway_network_object = base_roadway_net,
    network_variable = 'lanes'
)

base_roadway_net.links_df.lanes.value_counts()

2020-10-21 13:54:43, INFO: Determining number of lanes
2020-10-21 13:54:43, INFO: Determining number of lanes
2020-10-21 14:01:15, INFO: Finished determining number of lanes using variable: lanes
2020-10-21 14:01:15, INFO: Finished determining number of lanes using variable: lanes


1    2072663
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: lanes, dtype: int64

In [23]:
pd.crosstab(base_roadway_net.links_df.roadway, base_roadway_net.links_df.lanes)

lanes,1,2,3,4,5,6,7,8,9
roadway,,,,,,,,,
cycleway,99067,36,4,0,0,0,0,0,0
footway,247365,87,27,8,2,0,0,0,0
maz,531308,0,0,0,0,0,0,0,0
motorway,115,576,640,1026,314,96,6,1,0
motorway_link,3561,1184,187,44,7,0,0,0,0
primary,2555,11633,7466,3151,1367,235,71,14,0
primary_link,1356,110,35,3,2,0,0,0,0
residential,503743,33033,1424,369,90,7,0,0,0
secondary,25600,25120,7300,6494,3534,435,44,2,0


# Write Out

## As pickle

In [24]:
roadway_filename = os.path.join(output_dir, 'base_roadway_network.pickle')
pickle.dump(base_roadway_net, open(roadway_filename, 'wb'))